In [33]:
%load_ext autoreload
%autoreload 2

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
df = pd.read_csv('../data/raw/PJME_hourly.csv', parse_dates=['Datetime'])

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145366 entries, 0 to 145365
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Datetime  145366 non-null  datetime64[ns]
 1   PJME_MW   145366 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.2 MB


,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [35]:
df = df.sort_values("Datetime")
print(df["Datetime"].is_monotonic_increasing)   # should be True
print(df["Datetime"].duplicated().sum())

True
4


In [36]:
df.isna().sum()

Datetime    0
PJME_MW     0
dtype: int64

In [37]:
# we have 4 duplicated rows
# find duplicated timestamps
dupes = df[df["Datetime"].duplicated(keep=False)]

# show them grouped by timestamp
dupes.groupby("Datetime")["PJME_MW"].apply(list)

Datetime
2014-11-02 02:00:00    [23755.0, 22935.0]
2015-11-01 02:00:00    [21567.0, 21171.0]
2016-11-06 02:00:00    [20795.0, 21692.0]
2017-11-05 02:00:00    [21236.0, 20666.0]
Name: PJME_MW, dtype: object

In [38]:
# the values are different, so will just average
df = df.groupby("Datetime", as_index=False)["PJME_MW"].mean()

print(df["Datetime"].duplicated().sum())
df

0


,Datetime,PJME_MW
0,2002-01-01 01:00:00,30393.0
1,2002-01-01 02:00:00,29265.0
2,2002-01-01 03:00:00,28357.0
3,2002-01-01 04:00:00,27899.0
4,2002-01-01 05:00:00,28057.0
...,...,...
145357,2018-08-02 20:00:00,44057.0
145358,2018-08-02 21:00:00,43256.0
145359,2018-08-02 22:00:00,41552.0
145360,2018-08-02 23:00:00,38500.0


In [39]:
df = df.set_index("Datetime").sort_index()
df

,PJME_MW
Datetime,
2002-01-01 01:00:00,30393.0
2002-01-01 02:00:00,29265.0
2002-01-01 03:00:00,28357.0
2002-01-01 04:00:00,27899.0
2002-01-01 05:00:00,28057.0
...,...
2018-08-02 20:00:00,44057.0
2018-08-02 21:00:00,43256.0
2018-08-02 22:00:00,41552.0


In [43]:
df.to_parquet("../data/interim/pjme_hourly.parquet", index=True)

In [45]:
pd.read_parquet("../data/interim/pjme_hourly.parquet").info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145362 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145362 non-null  float64
dtypes: float64(1)
memory usage: 2.2 MB


### Aggregate to daily:

In [47]:
df_daily = df.resample("D").sum()
df_daily.to_parquet("../data/interim/pjme_daily.parquet")

In [48]:
df_daily

,PJME_MW
Datetime,
2002-01-01,714857.0
2002-01-02,822277.0
2002-01-03,828285.0
2002-01-04,809171.0
2002-01-05,729723.0
...,...
2018-07-30,790978.0
2018-07-31,828938.0
2018-08-01,941539.0


In [49]:
df_daily.to_period(freq='D')

,PJME_MW
Datetime,
2002-01-01,714857.0
2002-01-02,822277.0
2002-01-03,828285.0
2002-01-04,809171.0
2002-01-05,729723.0
...,...
2018-07-30,790978.0
2018-07-31,828938.0
2018-08-01,941539.0
